In [5]:
import pandas as pd
import datetime
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import GridSearchCV

## Read in the final version of the data that has the weather and census data merged in already and do some further processing

In [93]:
# read in the data

modeldata_df = pd.read_csv("../static/dataignore/modeldata.csv")


C:\Users\jeickholt\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [110]:
# assign to a dataframe

data_final_df=modeldata_df

In [100]:
# # look at the available columns

# print(data_final_df.columns.values)


['case number' 'sr location' 'county' 'district' 'neighborhood' 'tax id'
 'trash quad' 'recycle quad' 'trash day' 'heavy trash day' 'recycle day'
 'key map' 'management district' 'department' 'division' 'sr type' 'queue'
 'sla' 'status' 'sr create date' 'due date' 'date closed' 'overdue'
 'title' 'latitude' 'longitude' 'channel type' 'date' 'year' 'month'
 'season' 'zipcode' 'daystoclose' 'daysdue' 'missedduedate' 'maxtemp'
 'avgtemp' 'mintemp' 'precip' 'week' 'weekmaxtemp' 'weekavgtemp'
 'weekmintemp' 'weekprecip' 'Population' 'Median Age' 'Household Income'
 'Per Capita Income' 'Poverty Rate' 'Total Households'
 'Total Owner Occupied' '% Owner Occupied']


In [9]:
# ################################################################
# # create a date value formatted the same as the weather data

# splitdate = data_final_df["sr create date"].str.split(" ", n=1, expand=True)

# data_final_df["create_date"] = splitdate[0]
# data_final_df["create_date"] = pd.to_datetime(data_final_df["create_date"])
# data_final_df["create_date"] = data_final_df['create_date'].dt.strftime('%Y-%m-%d')


# # I set these as date formats using the to_datetime

# data_final_df["create_date_time"] = pd.to_datetime(data_final_df["sr create date"])
# data_final_df["date_closed_time"] = pd.to_datetime(data_final_df["date closed"])
# data_final_df["due_date_time"] = pd.to_datetime(data_final_df["due date"])


# # here  I define weather is was closed by the due date (0) or not (1)
# data_final_df["missed_due_date"] = np.where(data_final_df["due_date_time"] > data_final_df["date_closed_time"], 0, 1)

# # calculate the difference between the closed date at the create date
# data_final_df["time_to_close"] = data_final_df["date_closed_time"] - data_final_df["create_date_time"]

# # apply formatting so that is is in seconds. then calculate the number of days.
# data_final_df["seconds_to_close"] = data_final_df["time_to_close"].astype('timedelta64[s]')
# data_final_df["days_to_close"] = data_final_df["seconds_to_close"]/60/60/24
 
# # data311_df[["create_date_time", "date_closed_time", "time_to_close","seconds_to_close", "days_to_close"]]


# data_final_df[["create_date_time","due_date_time", "date_closed_time", "missed_due_date", "days_to_close"]].head()



,create_date_time,due_date_time,date_closed_time,missed_due_date,days_to_close
0,2017-01-01 00:14:04,2017-01-02 00:14:04,2017-01-01 00:23:57,0,0.006863
1,2017-01-01 00:30:48,2017-01-11 00:30:47,2017-02-02 13:20:06,1,32.534236
2,2017-01-01 04:26:27,2017-01-02 04:26:27,2017-01-01 07:55:51,0,0.145417
3,2017-01-01 05:22:37,2017-01-02 05:22:35,2017-01-04 11:00:04,1,3.234340
4,2017-01-01 05:23:33,2017-01-03 05:23:33,2017-01-04 03:40:04,1,2.928137


In [97]:
# keep just the fields that will be used in analysis

# data_final_df=data_final_df[["create_date_time","create_date","days_to_close","missed_due_date","sr type","tempMax","precipitation","Population","Median Age",
#                     "Household Income","Poverty Rate","% Owner Occupied"]]


# data_final_df["sr type"].value_counts()


# # There were some entries where we were not able to get the zipcode, thus we weren't able to map in the 
# # weather data and census data, so dropping these in able to do modeling

# data_final_df = data_final_df.dropna()
# data_final_df.isna().sum()


In [98]:
# # calculate the weekly average precipitation and temperature

# data_final_df['Year-Week'] = data_final_df['create_date_time'].dt.strftime('%Y-%U')

# data_final_df[["create_date","Year-Week"]]

# # group by the year, week and calculate the avereage

# group_weather_df = data_final_df[["Year-Week","tempMax","precipitation"]].groupby('Year-Week').mean()

# group_weather_df = group_weather_df.rename(columns={'tempMax': 'tempAvg', 'precipitation': 'precipAvg'})

# group_weather_df.head()

In [99]:
# # merge the weekly average weather back in and drop the daily data 

# data_final_df = data_final_df.merge(group_weather_df, how="left", left_on="Year-Week", right_on="Year-Week")

# data_final_df = data_final_df.drop(columns=["tempMax","precipitation"])

# data_final_df.head()

In [137]:
data_final_df.describe()

# data_final_df[data_final_df["days_to_close"]>500].shape

data_final_df.shape
print(data_final_df.columns.values)


# keep just the fields that will be used in analysis

data_final_df=data_final_df[["daystoclose","missedduedate","sr type","avgtemp","weekprecip","Population","Median Age",
                    "Household Income","Poverty Rate","% Owner Occupied"]]

print(data_final_df.shape)
data_final_df.head()

['daystoclose' 'missedduedate' 'sr type' 'avgtemp' 'weekprecip'
 'Population' 'Median Age' 'Household Income' 'Poverty Rate'
 '% Owner Occupied']
(661926, 10)


,daystoclose,missedduedate,sr type,avgtemp,weekprecip,Population,Median Age,Household Income,Poverty Rate,% Owner Occupied
0,0.006863,0,Traffic Signal Maintenance,67.5,0.265714,38931.0,31.9,44957.0,22.824998,24.946453
1,32.534236,1,Water Leak,67.5,0.265714,21732.0,39.1,107003.0,5.268728,49.179798
2,0.145417,0,Traffic Signal Maintenance,67.5,0.265714,915.0,44.6,250001.0,6.666667,8.524590
3,3.234340,1,Sewer Wastewater,67.5,0.265714,27364.0,34.1,30164.0,29.399942,48.344736
4,2.928137,1,Water Service,67.5,0.265714,27364.0,34.1,30164.0,29.399942,48.344736


In [138]:
# Some zipcodes could not be mapped in, so we were not able to map in the census data
# Dropping these to enable modeling

data_final_df=data_final_df.dropna()
print(data_final_df.shape)

(660378, 10)


In [139]:

# need to create dummy variable for the "sr type" column, which is categorical

data_dummies_df=pd.get_dummies(data_final_df["sr type"])



# add dummy variables in to dataframe with other variables

data_regress_df=pd.concat([data_final_df, data_dummies_df], axis=1)  

data_regress_df = data_regress_df[["Population", "Median Age", "Household Income", "Poverty Rate", "% Owner Occupied",
                            "avgtemp","weekprecip",       
                            "Container Problem", "Drainage", "Missed Garbage Pickup", "Missed Heavy Trash Pickup", 
                            "Missed Recycling Pickup", "Nuisance On Property", "SWM Escalation", "Sewer Wastewater", 
                            "Storm Debris Collection", "Street Condition", "Street Hazard", "Traffic Signal Maintenance", 
                            "Traffic Signs", "Water Leak", "Water Service"]]

# create y variable for the regression analysis.  Ended up not using the regression analysis

y=data_final_df["daystoclose"].values.reshape(-1,1)


# create x variables (drop columns that aren't needed)

X_var=data_regress_df
X_var.isna().sum()
 
print(X_var.shape)
print(X_var.columns.values)
X_var.head()  




(660378, 22)
['Population' 'Median Age' 'Household Income' 'Poverty Rate'
 '% Owner Occupied' 'avgtemp' 'weekprecip' 'Container Problem' 'Drainage'
 'Missed Garbage Pickup' 'Missed Heavy Trash Pickup'
 'Missed Recycling Pickup' 'Nuisance On Property' 'SWM Escalation'
 'Sewer Wastewater' 'Storm Debris Collection' 'Street Condition'
 'Street Hazard' 'Traffic Signal Maintenance' 'Traffic Signs' 'Water Leak'
 'Water Service']


,Population,Median Age,Household Income,Poverty Rate,% Owner Occupied,avgtemp,weekprecip,Container Problem,Drainage,Missed Garbage Pickup,...,Nuisance On Property,SWM Escalation,Sewer Wastewater,Storm Debris Collection,Street Condition,Street Hazard,Traffic Signal Maintenance,Traffic Signs,Water Leak,Water Service
0,38931.0,31.9,44957.0,22.824998,24.946453,67.5,0.265714,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,21732.0,39.1,107003.0,5.268728,49.179798,67.5,0.265714,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,915.0,44.6,250001.0,6.666667,8.524590,67.5,0.265714,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,27364.0,34.1,30164.0,29.399942,48.344736,67.5,0.265714,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,27364.0,34.1,30164.0,29.399942,48.344736,67.5,0.265714,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [140]:
# define new data for feeding in to models

# Population, Median Age, Household Income, Poverty Rate, % Owner Occupied, tempAvg, precipAvg, Container Problem, Drainage,
# Missed Garbage Pickup, Missed Heavy Trash Pickup, Missed Recycling Pickup, Nuisance On Property, SWM Escalation, 
# Sewer Wasterwater, Storm Debris Collection, Street Condition,
# Street Hazard, Traffic Signal Maintenance, Traffic Sign, Water Leak, Water Service

new_data = [[38931, 31.9, 44957, 22.82, 24.95, 56.07, 0.27, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

## Build Logistic Regression Model to predict if issue was not closed by due date

#### Define the model

In [154]:
# I need to define y as the classfication variable.  The X_var can stay the same

y_class = data_final_df["missedduedate"]

y_class.value_counts()

0    460331
1    200047
Name: missedduedate, dtype: int64

In [142]:
# define logistic model (note a grid search suggested the C and penalty values used)
# Note that changing the parameters based on the output of the grid search didn't improve the model very much

from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(C=166, penalty='l1')
classifier

LogisticRegression(C=166, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l1',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

#### Define training and test data

In [143]:
# define training and test sets

from sklearn.model_selection import train_test_split


X_train, X_test, y_class_train, y_class_test = train_test_split(X_var, y_class, random_state=33)


#### Scale the data

In [144]:
# define standard scaler

from sklearn.preprocessing import StandardScaler

X_scaler = StandardScaler().fit(X_train)


In [145]:
# scale the x data (y is just 0 or 1, so not scaling)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

#### Fit the model

In [146]:
# fit the model

classifier.fit(X_train_scaled, y_class_train)

C:\Users\jeickholt\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=166, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l1',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [147]:
# look at the scores

print(f"Training Data Score: {classifier.score(X_train_scaled, y_class_train)}")
print(f"Testing Data Score: {classifier.score(X_test_scaled, y_class_test)}")

Training Data Score: 0.7293951135007662
Testing Data Score: 0.729065083739665


#### Examine predictions in the test data

In [148]:
# look at the prediction vs actual for the test data

predictions = classifier.predict(X_test_scaled)
probs = classifier.predict_proba(X_test_scaled)


pd.DataFrame({
    "Prediction": predictions, 
    "Actual": y_class_test, 
    "On Time": [p[0] for p in probs], 
    "Overdue": [p[1] for p in probs]})

,Prediction,Actual,On Time,Overdue
546026,0,0,0.856677,0.143323
472932,0,0,0.654441,0.345559
410173,1,1,0.439069,0.560931
296908,0,0,0.870378,0.129622
226694,0,1,0.662966,0.337034
141832,0,1,0.657869,0.342131
245206,1,0,0.441943,0.558057
92224,0,0,0.820795,0.179205
314573,0,0,0.975919,0.024081
66778,0,0,0.923321,0.076679


#### Predict a new data point

In [168]:
# predict a new set of data

# Population, Median Age, Household Income, Poverty Rate, % Owner Occupied, tempAvg, precipAvg, Container Problem, Drainage,
# Missed Garbage Pickup, Missed Heavy Trash Pickup, Missed Recycling Pickup, Nuisance On Property, SWM Escalation, 
# Sewer Wasterwater, Storm Debris Collection, Street Condition,
# Street Hazard, Traffic Signal Maintenance, Traffic Sign, Water Leak, Water Service

new_data = [[38931, 31.9, 44957, 25.82, 24.95, 56.07, 0.27, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

new_data_scaled = X_scaler.transform(new_data)

new_predict = classifier.predict(new_data_scaled)
new_prob = classifier.predict_proba(new_data_scaled)


print("New Prediction: %.3f" % new_predict)

print("Prob of Meeting Deadline : %.3f" % new_prob[0,0])
print("Prob of Not Meeting Deadline : %.3f" % new_prob[0,1])
    

New Prediction: 1.000
Prob of Meeting Deadline : 0.443
Prob of Not Meeting Deadline : 0.557


### Do a grid search to try and find a better model.

In [150]:
logistic = LogisticRegression()

In [151]:
# Create regularization penalty space
penalty = ['l1', 'l2']

# Create regularization hyperparameter space
C = np.logspace(0, 4, 10)

# Create hyperparameter options
hyperparameters = dict(C=C, penalty=penalty)

In [152]:
# Create grid search using 5-fold cross validation
clf = GridSearchCV(logistic, hyperparameters, cv=5, verbose=0)

In [153]:
# Fit grid search
best_model = clf.fit(X_train_scaled, y_class_train)

C:\Users\jeickholt\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\jeickholt\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\jeickholt\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\jeickholt\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\jeickholt\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a

C:\Users\jeickholt\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\jeickholt\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\jeickholt\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\jeickholt\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\jeickholt\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a

C:\Users\jeickholt\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\jeickholt\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\jeickholt\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\jeickholt\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\jeickholt\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a

In [133]:
# View best hyperparameters
print('Best Penalty:', best_model.best_estimator_.get_params()['penalty'])
print('Best C:', best_model.best_estimator_.get_params()['C'])

Best Penalty: l1
Best C: 166.81005372000593


#### Output logistic model with pickle

In [89]:
# output model to pickle

import pickle

with open('../static/models/logistic_model.pkl','wb') as f:
    pickle.dump(classifier, f)

## Develop KNN model to predict if issue was not closed by due date

#### Fit the model

In [23]:
from sklearn.neighbors import KNeighborsClassifier

In [84]:
# Chose k=4 after running the loop below to check other values for k

knn = KNeighborsClassifier(n_neighbors=4)
knn.fit(X_train_scaled, y_class_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=4, p=2,
                     weights='uniform')

#### Check the score

In [85]:
print('k=5 Test Acc: %.3f' % knn.score(X_train_scaled, y_class_train))
knn

k=5 Test Acc: 0.843


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=4, p=2,
                     weights='uniform')

In [92]:
# look at the prediction vs actual for the test data

predictions_knn = knn.predict(X_test_scaled)
probs_knn = knn.predict_proba(X_test_scaled)


pd.DataFrame({
    "Prediction": predictions_knn, 
    "Actual": y_class_test, 
    "On Time": [p[0] for p in probs_knn], 
    "Overdue": [p[1] for p in probs_knn]})

,Prediction,Actual,On Time,Overdue
41449,0,0,1.00,0.00
307201,0,0,1.00,0.00
48134,1,0,0.25,0.75
390753,0,0,1.00,0.00
147945,0,0,0.75,0.25
182701,0,1,1.00,0.00
335712,0,0,0.75,0.25
372443,1,1,0.00,1.00
506217,0,0,1.00,0.00
160745,0,0,0.75,0.25


#### Predict a new data point

In [91]:
# predict a  new data point

# Population, Median Age, Household Income, Poverty Rate, % Owner Occupied, tempAvg, precipAvg, Container Problem, Drainage,
# Missed Garbage Pickup, Missed Heavy Trash Pickup, Missed Recycling Pickup, Nuisance On Property, SWM Escalation, 
# Sewer Wasterwater, Storm Debris Collection, Street Condition,
# Street Hazard, Traffic Signal Maintenance, Traffic Sign, Water Leak, Water Service


new_data_knn = [[38931, 31.9, 44957, 22.82, 24.95, 56.07, 0.27, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

# have to scale the new data
new_data_knn_scaled = X_scaler.transform(new_data_knn)

new_predict_knn = knn.predict(new_data_knn_scaled)

print("New Prediction: %.3f" % new_predict_knn)

new_prob_knn = knn.predict_proba(new_data_knn_scaled)

print("Prob of Meeting Deadline : %.3f" % new_prob_knn[0,0])
print("Prob of Not Meeting Deadline : %.3f" % new_prob_knn[0,1])

print(new_data_knn)
print(new_data_knn_scaled)
    
    
parameters = knn.get_params



New Prediction: 0.000
Prob of Meeting Deadline : 1.000
Prob of Not Meeting Deadline : 0.000
[[38931, 31.9, 44957, 22.82, 24.95, 56.07, 0.27, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[ 0.39366916 -0.57383356 -0.35805553  0.21805589 -1.95933636 -2.13800639
   0.24057979  2.81738006 -0.21252576 -0.37613373 -0.27086858 -0.30086824
  -0.30896869 -0.20270583 -0.33567591 -0.13876926 -0.1752962  -0.22708341
  -0.2092351  -0.19037391 -0.35690846 -0.2390681 ]]


In [22]:
# This output a file that is >200 MB, so not running it.

# output model using pickle

# import pickle

# with open('../static/models/knn_model.pkl','wb') as f:
#     pickle.dump(knn, f)

### Look at different number of neighbors to see if a better model can be found
#### This took a long time to run, given the large amount of data.  I stopped it after K=11 (after running 2 hours), since it was clearly not improving.  It looks like k=4 is probably about right.

In [ ]:
# Loop through different k values to see which has the highest accuracy
# Note: We only use odd numbers because we don't want any ties
train_scores = []
test_scores = []

for k in range(1, 20, 2):
    # create the classifier for each value of K
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train_scaled, y_class_train)
    
    #Score with both test and train data
    train_score = knn.score(X_train_scaled, y_class_train)
    test_score = knn.score(X_test_scaled, y_class_test)
    
    #append the scores to our list of scores
    train_scores.append(train_score)
    test_scores.append(test_score)
    print(f"k: {k}, Train/Test Score: {train_score:.3f}/{test_score:.3f}")
    

k: 1, Train/Test Score: 0.830/0.768
k: 3, Train/Test Score: 0.847/0.798
k: 5, Train/Test Score: 0.845/0.804
k: 7, Train/Test Score: 0.840/0.804
k: 9, Train/Test Score: 0.834/0.804
k: 11, Train/Test Score: 0.829/0.801
